In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets

In [ ]:
from transformers import pipeline
bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=bert_ckpt)
query = """Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th in Paris and I need a 15 passenger van"""
pipe(query)

In [ ]:
class PerformanceBenchmark:
  def __init__(self,pipeline,dataset,optim_type="BERT baseline"):
    self.pipeline=pipeline
    self.dataset=datset
    self.optim_type=optim_type
  def compute_accuracy(self):
    pass
  def compute_size(self):
    pass
  def time_pipeline(self):
    pass
  def run_benchmark(self):
    metrics={}
    metrics[self.optim_type]=self.compute_size()
    metrics[self.optim_type].update(self.time_pipeline())
    metrics[self.optim_type].update(self.compute_accuracy())
    return metrics

In [ ]:
!pip install dataset

In [ ]:
from datasets import load_dataset
clinc = load_dataset("clinc_oos", "plus")
sample = clinc["test"][42]
sample

In [ ]:
intents = clinc["test"].features["intent"]
intents.int2str(sample["intent"])

In [ ]:
from datasets import load_metric
accuracy_score = load_metric("accuracy")

In [ ]:
def compute_accuracy(self):
	"""This overrides the PerformanceBenchmark.compute_accuracy() method"""
	preds, labels = [], []
	for example in self.dataset:
		pred = self.pipeline(example["text"])[0]["label"]
		label = example["intent"]
		preds.append(intents.str2int(pred))
		labels.append(label)
        accuracy = accuracy_score.compute(predictions=preds, references=labels)
        print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")
     return accuracy PerformanceBenchmark.compute_accuracy = compute_accuracy


In [ ]:
list(pipe.model.state_dict().items())[42]

In [ ]:
torch.save(pipe.model.state_dict(), "model.pt")

In [ ]:
import torch
from pathlib import Path
def compute_size(self):
	"""This overrides the PerformanceBenchmark.compute_size() method"""
	state_dict = self.pipeline.model.state_dict()
	tmp_path = Path("model.pt")
	torch.save(state_dict, tmp_path)
	# Calculate size in megabytes
    size_mb = Path(tmp_path).stat().st_size / (1024 * 1024)
    # Delete temporary file
    tmp_path.unlink()
    print(f"Model size (MB) - {size_mb:.2f}")
    return {"size_mb": size_mb}

 PerformanceBenchmark.compute_size = compute_size


In [ ]:
from time import perf_counter
for _ in range(3):
	start_time = perf_counter()
	_ = pipe(query)
	latency = perf_counter() - start_time
	print(f"Latency (ms) - {1000 * latency:.3f}")

In [ ]:
import numpy as np
def time_pipeline(self,query="What is the pin number for my account?"):
  latencies=[]
  for _ in range(10):
    _=self.pipeline(query)
  for _ in range(100):
    start_time=perf_counter()
    _=self.pipeline(query)
    latency=perf_counter()-start_time
    latencies.append(latency)
  time_avg_ms=1000*np.mean(latencies)
  time_std_ms=1000*np.std(latencies)
  print(f"Average latency (ms) - {time_avg_ms:.2f}+\-{time_std_ms:.2f}")
  return {'time_avg_ms':time_avg_ms,"time_std_ms":time_std_ms}

PerformanceBenchmark.time_pipeline=time_pipeline


In [ ]:
pb = PerformanceBenchmark(pipe, clinc["test"])
perf_metrics = pb.run_benchmark()

In [ ]:
from transformers import TrainingArguments
class DistillationTrainingArguments(TrainingArguments):
	def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
		super().__init__(*args, **kwargs)
		self.alpha = alpha
        self.temperature = temperature


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer
class DistilltionTrainer(Trainer):
  def __init__(self,*args,teacher_method=None,**kwargs):
    super().__init__(*args,**kwargs)
    self.teacher_method=teacher_method
  def comput_loss(self,metjod,inputs,return_outputs=False):
    outputs_stu=method(**inputs)
    loss_ce=outputs_stu.loss
    logits_stu=outputs_stu.logits
    with torch.no_grad():
      outputs_tea=self.teacher_method(**inputs)
      logits_tea=outputs_tea.logits
    loss_fact=nn.KLDivLoss(reduction="batchmean")
    loss_kd=self,args.temperature**2*loss_fact(
        F.log_softmax(logits_stu/self.args.temperature,dim=-1),
        F.softmax(logits_tea/self.args.temperature,dim=-1)
    )
  loss=self.args.alpha*loss_ce+(1.-self.args.alpha)*loss_kd
  return (loss,outputs_stu) if return_outputs else loss

In [ ]:
from transformers import AutoTokenizer
student_ckpt="distilbert-base-uncased"
student_tokenizer=AutoTokenizer.from_pretrained(student_ckpt)
def tokenize_text(batch):
  return student_tokenizer(batch['text'],truncation=True)

clinc_enc=clinc,map(tokenize_text,batched=True,remove_columns=['text'])
clinc_enc=clinc_enc.rename_column('intent','labels')

In [ ]:
def compute_metrics(pred):
	predictions, labels = pred
	predictions = np.argmax(predictions, axis=1)
	return accuracy_score.compute(predictions=predictions, references=labels)


In [ ]:
batch_size = 48
finetuned_ckpt = "distilbert-base-uncased-finetuned-clinc"
student_training_args = DistillationTrainingArguments(
    output_dir=finetuned_ckpt,
    evaluation_strategy = "epoch",
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    alpha=1,
    weight_decay=0.01)


In [ ]:
id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

In [ ]:
from transformers import AutoConfig
num_labels = intents.num_classes
student_config = (AutoConfig .from_pretrained(student_ckpt, num_labels=num_labels, id2label=id2label, label2id=label2id))


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def student_init():
	return (AutoModelForSequenceClassification .from_pretrained(student_ckpt, config=student_config).to(device))


In [ ]:
teacher_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
teacher_model = (AutoModelForSequenceClassification .from_pretrained(teacher_ckpt, num_labels=num_labels) .to(device))
distilbert_trainer = DistillationTrainer(model_init=student_init, teacher_model=teacher_model, args=student_training_args, train_dataset=clinc_enc['train'], eval_dataset=clinc_enc['validation'], compute_metrics=compute_metrics, tokenizer=student_tokenizer)

distilbert_trainer.train()


In [ ]:
finetuned_ckpt = "transformersbook/distilbert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=finetuned_ckpt)


In [ ]:
optim_type = "DistilBERT"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
def plot_metrics(perf_metrics,current_optim_type):
  df=pd.DataFrame.form_dict(perf_metrics,orient='index')

  for idx in df.index:
    df_opt=df.loc[idx]
    if idx==current_optim_type:
      plt.scatter(df_opt['time_avg_ms'],df_opt['accuracy']*100
                  alpha=0.5,s=df_opt['size_mb'],label=idx,marker='$\u25CC$')
    else:
      plt.scatter(df_opt['time_avg_ms'],df_opt['accuracy']*100,s=df_opt['size_mb'],label=idx,alpha=0.5)

  legend=plt.legend(bbox_to_anchor=(1,1))
  for handle in legend.legendHandles:
    handle.set_size([20])

  plt.ylim(80,90)
  xlim=int(perf_metrics['BERT baseline']['time_avg_ms']+3)
  plt.xlim(1,xlim)
  plt.ylable('Accuracy(%)')
  plt.xlabel('Average latency(ms)')
  plt.show()
plot_metrics(perf_metrics,optim_type)

In [ ]:
def objective(trial):
	x = trial.suggest_float("x", -2, 2)
	y = trial.suggest_float("y", -2, 2)
	return (1 - x) ** 2 + 100 * (y - x ** 2) ** 2


In [ ]:
import optuna
study = optuna.create_study()
study.optimize(objective, n_trials=1000)


In [ ]:
study.best_params

In [ ]:
def hp_space(trial):
	return {"num_train_epochs": trial.suggest_int("num_train_epochs", 5, 10), "alpha": trial.suggest_float("alpha", 0, 1), "temperature": trial.suggest_int("temperature", 2, 20)}


In [ ]:
best_run = distilbert_trainer.hyperparameter_search( n_trials=20, direction="maximize", hp_space=hp_space)
print(best_run)

In [ ]:
for k,v in best_run.hyperparameters.items():
  setattr(student_training_args,k,v)

distilled_ckpt='distilbert-base-uncased-distilled-clinc'

student_training_args.output_dir=distilled_ckpt

distil_trainer=DistillationTrainer(model_init=student_init,
        teacher_model=teacher_method,args=student_training_args,
        train_dataset=clinc_enc['train'],eval_dataset=clinc_enc['validation'],
        compute_metrics=compute_metrics,tokenizer=student_tokenizer)

distil_trainer.train()                                         )

In [ ]:
distilled_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
pipe = pipeline("text-classification", model=distilled_ckpt)
optim_type = "Distillation"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())
plot_metrics(perf_metrics, optim_type)

In [ ]:
import matplotlib.pyplot as plt
state_dict = pipe.model.state_dict()
weights = state_dict["distilbert.transformer.layer.0.attention.out_lin.weight"]

plt.hist(weights.flatten().numpy(), bins=250, range=(-0.3,0.3), edgecolor="C0")

plt.show()


In [ ]:
zero_point = 0
scale = (weights.max() - weights.min()) / (127 - (-128))


In [ ]:
(weights / scale + zero_point).clamp(-128, 127).round().char()

In [ ]:
from torch import quantize_per_tensor
dtype = torch.qint8
quantized_weights = quantize_per_tensor(weights, scale, zero_point, dtype)
quantized_weights.int_repr()

In [ ]:
%%timeit
weights @ weights
393 μs ± 3.84 μs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
from torch.nn.quantized import QFunctional

q_fn = QFunctional()

In [ ]:
%%timeit
q_fn.mul(quantized_weights, quantized_weights)
23.3 μs ± 298 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
import sys
sys.getsizeof(weights.storage()) / sys.getsizeof(quantized_weights.storage())


In [ ]:
from torch.quantization import quantize_dynamic
model_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = (AutoModelForSequenceClassification .from_pretrained(model_ckpt).to("cpu"))
model_quantized = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)


In [ ]:
pipe = pipeline("text-classification", model=model_quantized, tokenizer=tokenizer)
optim_type = "Distillation + quantization"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())

Model size (MB) - 132.40
Average latency (ms) - 12.54 +\- 0.73
Accuracy on test set - 0.876

plot_metrics(perf_metrics, optim_type)


In [ ]:
import os
from psutil import cpu_count
os.environ["OMP_NUM_THREADS"] = f"{cpu_count()}"
os.environ["OMP_WAIT_POLICY"] = "ACTIVE"


In [ ]:
from transformers.convert_graph_to_onnx
import convert
model_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
onnx_model_path = Path("onnx/model.onnx")
convert(framework="pt", model=model_ckpt, tokenizer=tokenizer, output=onnx_model_path, opset=12, pipeline_name="text-classification")


In [ ]:
from onnxruntime import (GraphOptimizationLevel, InferenceSession, SessionOptions)
def create_model_for_provider(model_path, provider="CPUExecutionProvider"):
	options = SessionOptions()
	options.intra_op_num_threads = 1
	options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
	session = InferenceSession(str(model_path), options, providers=[provider])
	session.disable_fallback()
	return session

onnx_model = create_model_for_provider(onnx_model_path)


In [ ]:
inputs = clinc_enc["test"][:1]
del inputs["labels"] l
ogits_onnx = onnx_model.run(None, inputs)[0]
logits_onnx.shape


In [ ]:
np.argmax(logits_onnx)

In [ ]:
clinc_enc["test"][0]["labels"]

In [ ]:
from scipy.special import softmax
class OnnxPipeline:
	def __init__(self, model, tokenizer):
		self.model = model
		self.tokenizer = tokenizer
	def __call__(self, query):
		model_inputs = self.tokenizer(query, return_tensors="pt")
		inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()}
		logits = self.model.run(None, inputs_onnx)[0][0, :]
		probs = softmax(logits)
		pred_idx = np.argmax(probs).item()
		return [{"label": intents.int2str(pred_idx), "score": probs[pred_idx]}]


In [ ]:
pipe = OnnxPipeline(onnx_model, tokenizer) pipe(query)

In [ ]:
class OnnxPerformanceBenchmark(PerformanceBenchmark):
	def __init__(self, *args, model_path, **kwargs):
		super().__init__(*args, **kwargs)
		self.model_path = model_path
	def compute_size(self):
		size_mb = Path(self.model_path).stat().st_size / (1024 * 1024)
		print(f"Model size (MB) - {size_mb:.2f}")
		return {"size_mb": size_mb}


In [ ]:
optim_type = "Distillation + ORT"
pb = OnnxPerformanceBenchmark(pipe, clinc["test"], optim_type, model_path="onnx/model.onnx")
perf_metrics.update(pb.run_benchmark())
Model size (MB) - 255.88
Average latency (ms) - 21.02 +\- 0.55
Accuracy on test set - 0.868

plot_metrics(perf_metrics, optim_type)


In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType
model_input = "onnx/model.onnx"
model_output = "onnx/model.quant.onnx"
quantize_dynamic(model_input, model_output, weight_type=QuantType.QInt8)


In [ ]:
nnx_quantized_model = create_model_for_provider(model_output)
pipe = OnnxPipeline(onnx_quantized_model, tokenizer)
optim_type = "Distillation + ORT (quantized)"
pb = OnnxPerformanceBenchmark(pipe, clinc["test"], optim_type, model_path=model_output)
perf_metrics.update(pb.run_benchmark()